# Přednáška 5: Zobecněné lineární modely (GLM, Generalized Linear Models)

**Obsah přednášky**
- Motivační příklad
- GLM - základní principy
- logistická regrese
- sigmoid fce, logistická fce (scipy)
- kvantifikace chyb
- příklad

## GLM - trocha matematiky 

Označme stejně jako v případě dříve probraných **lineárních modelů** pozorovanou náhodnou veličinu $Y_t$ s realizacemi $y_1, y_2, \ldots$. Budeme předpokládat, že má nějaké rozdělení z **exponenciální třídy distribucí**, což jak už rovněž víme, může být například normální, binomické, Poissonovo nebo gama. Stejně jako u lineárních modelů budeme uvažovat **vysvětlující (nezávislou) proměnnou - regresor** $X_t$ s realizacemi $x_1, x_2,\ldots$ a dále **vektor regresních koeficientů** $\beta$ o stejném rozměru jako $X$.

Vzpomeňme na lineární regresi, kde $y_t = \beta^\intercal x_t + \varepsilon_t$.

> **Zobecněné lineární modely** jsou modely ve tvaru

$$
\mathbb{E}[Y_t] = g^{-1}(\beta^\intercal X_t),
$$

> kde $\mathbb{E}[Y_t]$ je střední hodnota $Y_t$, $\beta^\intercal X_t$ je lineární prediktor stejně jako v lineární regresi a $g$ je linková (spojovací) funkce.

Ještě lépe pochopitelné to bude po zmínce, že střední hodnota se někdy nazývá *očekávanou hodnotou*, angl. expected value nebo expectation.

Z rovnice výše plyne, že
$$
\beta^\intercal X_t = g(\mathbb{E}[Y_t]),
$$

tedy že linková funkce spojuje lineární prediktor $\beta^\intercal X_t$ se střední (očekávanou) hodnotou $Y_t$.

**Úkol:**
- lineární regrese je rovněž zobecněným lineárním modelem. Jak v tom případě vypadají funkce $g$ a $g^{-1}$?

### Příklady linkových funkcí $g$
#### Dichotomická veličina $Y \sim \{0, 1\}$
Pro binární (dichotomickou) veličinu s *Bernoulliho rozdělením* s pravděpodobností $p$ výhodou používáme funkci [logit](https://en.wikipedia.org/wiki/Logit):

$$
\mathit{logit}(p) = \log\left(\frac{p}{1-p}\right).
$$

Její inverze se nazývá [logistickou funkcí](https://en.wikipedia.org/wiki/Logistic_function), též sigmoid funkcí (to je ale spíše [celá množina funkcí](https://en.wikipedia.org/wiki/Logit)). Logistická funkce má tvar

$$
\mathit{logit}^{-1}(z) = \sigma(z) = \frac{e^z}{1+e^z} = \frac{1}{1+e^{-z}}.
$$

Za argument $z$ dosazujeme $\beta^\intercal x_t$, čímž dostáváme **logistickou regresi**. Grafy obou funkcí ukazují obrázky.

![logit](img/logit_expit.png)

#### Poissonovská veličina $Y_t \sim \textit{Poisson}(\cdot)$
Poissonovo rozdělení používáme pro popis počtu jevů nastalých v určitém místě nebo časovém okamžiku, např. počet částic dopadnuvších na detektor za daný časový okamžik, počet zákazníků kontaktujících call centrum v závislosti na velikosti lokality, počet úmrtí v dané věkové kategorii za definovaný časový úsek aj. Jelikož potřebujeme lineární prediktor $\beta^\intercal X_t$ zobrazit nějakým způsobem na množinu kladných čísel, můžeme s výhodou využít logaritmus. Inverzní zobrazení - počet realizací - nám pak dodá exponenciální funkce.

Jinými slovy: **Poissonovská regrese** pracuje s modelem

$$
\log \mathbb{E}[Y] = \beta^\intercal X, \qquad \text{respektive} \qquad \mathbb{E}[Y] = e^{\beta^\intercal X}
$$

## Logistická regrese

Logistický regresní model je určen pro popis dichotomické veličiny

$$
Y_t = 
\begin{cases} 
0 \quad \text{s pravděpodobností $p_t$}, \\ 
1 \quad \text{s pravděpodobností $1 - p_t$}.\\
\end{cases}
$$

Z dřívějška víme, že vhodnou distribucí pro $Y_t$ je distribuce **Bernoulliho** (též alternativní) s pravděpodobností $p_t$ jako svým parametrem,

$$
Y_t \sim \textit{Bernoulli}(p_t)
$$
a **se střední hodnotou** $\mathbb{E}[Y_t] = p_t$.

Máme-li k dispozici nezávislou vysvětlující proměnnou (regresor) $X_t$, pokusíme se najít vhodné $\beta$ a $Y_t$ s nimi vhodně provázat.

Připomeňme ještě, že pravděpodobnostní funkce Bernoulliho rozdělení je

$$
\begin{aligned}
f(y_t|p_t) &= p_t^{y_t} (1-p_t)^{1-y_t} \\
           &= f(y_t|x_t, \beta),
\end{aligned}
$$

kde druhý řádek jsme si doplnili pro pořádek, důvod se ukáže vzápětí :-)

Z teorie GLM výše víme, že potřebujeme vhodnou linkovou funkci, která prováže *lineární prediktor* $\beta^\intercal X_t$ s $Y_t$, respektive jeho střední (očekávanou) hodnotou,

$$
\mathbb{E}[Y_t|X_t, \beta] = g^{-1}(\beta^\intercal X_t).
$$

A dokonce už také víme, že vhodnou volbou pro $g$ je funkce $\textit{logit}$, a tedy

$$
\begin{aligned}
g(\mathbb{E}[Y_t|p_t]) 
&= g(\mathbb{E}[Y_t|X_t, \beta]) \\
&= g(p_t) \\
&= \textit{logit}(p_t) \\
&= \log \frac{p_t}{1-p_t}
= \beta^\intercal X_t. 
\end{aligned}
$$

Analogicky tedy inverzí
$$
\begin{aligned}
\mathbb{E}[Y_t|X_t, \beta]
&= p_t \\
&= \textit{logit}^{-1}(\beta^\intercal X_t) \\
&= \sigma(\beta^\intercal X_t) \\
&= \frac{1}{1+e^{-\beta^\intercal X_t}}.
\end{aligned}
$$

### Bayesovský odhad logistického regresního modelu

Bayesovský odhad $\beta$ není triviální, neboť aposteriorní distribuce

$$
\begin{aligned}
\pi(\beta|x_{0:t}, y_{0:t}) 
&\propto f(y_t|x_t, \beta) \times \pi(\beta|x_{0:t-1}, y_{0:t-1}) \\
&\propto p_t^{y_t} (1-p_t)^{1-y_t}) \times \pi(\beta|x_{0:t-1}, y_{0:t-1}) \\
&\propto [\sigma(\beta^\intercal x_t)]^{y_t}
[1-\sigma(\beta^\intercal x_t)]^{1-y_t} \times \pi(\beta|x_{0:t-1}, y_{0:t-1})
\end{aligned}
$$

není analyticky dostupná pro žádnou rozumnou volbu apriorna. Možností, jak k problému přistoupit je povícero - buď odhad pomocí metod Monte Carlo, nebo variačními metodami, nebo aproximativně laplaceovskou aproximací. My si vybereme tu poslední variantu.

Zvolíme si jako apriorno vícerozměrnou normální distribuci

$$
\beta|X_{0:t-1}, Y_{0:t-1} \sim \mathcal{N}(\widehat{\beta}_{t-1}, \Sigma_{t-1}). 
$$

Aposteriorní distribuci pořešíme aproximativně ve dvou krocích:
1. Najdeme maximum aposteriorní distribuce uvedené výše. Tím dostaneme MAP (maximum a posteriori) odhad.
2. Do tohoto maxima umístíme střední hodnotu normální hustoty a najdeme pro ni odpovídající vhodnou kovarianci.

**Úkoly:**
- obecně: jakým způsobem hledáme maximum funkce?

> **Intermezzo: Newtonova metoda**
>
> Je-li dána dostatečně "rozumná" funkce $g$ - zejm. spojitá s existující derivací a monotónní v určeném intervalu, pak pro nalezení bodu $x$ v němž platí  $g(x)=0$ lze použít Newtonovu metodu. Odstartujeme-li z "rozumného" bodu $x_0$, najdeme nový bod $x_1$ na průsečíku tečny v $g(x_0)$ s osou $x$ a takto postupně iterujeme až do dosažení požadované přesnosti nebo počtu iterací. Ve třech krocích nám nalezení $x_1$ vysvětlí následující obrázek.
![Newtonova metoda](img/Newton2.png)

**Úkoly:**
- Už víme, že budeme hledat maximum aposteriorní distribuce $\pi(\beta|x_{0:t}, y_{0:t})$. Co bude analogií k funkci $g$ v "Intermezzu"?

#### Hledání střední hodnoty aproximující normální distribuce
Už tedy víme, že budeme "newtonovsky" hledat maximum $\widehat{\beta}_t$. Můžeme to dělat i jen jednokrokovým Newtonem, kde za počáteční hodnotu (analogické k $x_0$) vybereme minulý odhad $\widehat{\beta}_{t-1}$,

$$
\widehat{\beta}_t = \widehat{\beta}_{t-1} - [l''(\widehat{\beta}_{t-1})]^{-1} l'(\widehat{\beta}_{t-1}),
$$

kde

$$
l(\widehat{\beta}_{t-1}) = \log \pi(\beta|x_{0:t}, y_{0:t}) 
$$

je logaritmus aposteriorní hustoty v bodě $\widehat{\beta}_{t-1}$. Pro úplnost dodejme, že

$$
\begin{aligned}
l'(\widehat{\beta}_{t-1}) 
&= x_t\left(y_t - \sigma(\widehat{\beta}_{t-1}x_t)\right),\\
l''(\widehat{\beta}_{t-1})
&= -\Sigma_{t-1} - x^\intercal R x \qquad\text{kde}\quad
R = \text{diag} \left[
\sigma\left(\widehat\beta_{t-1}^{\intercal}x_t \right)
\left(1-\sigma\left(\widehat\beta_{t-1}^{\intercal}x_t\right)\right)
\right].
\end{aligned}
$$

Tady se to trochu zkomplikovalo, neboť jsme museli dělat derivace ve více rozměrech.

#### Hledání kovariance aproximující normální hustoty
Pokud jsme nalezli MAP odhad, nafitujeme normální distribuci s kovariancí

$$
\Sigma_t = -[l''(\widehat{\beta}_{t-1})]^{-1}.
$$

To již máme spočteno z předchozího kroku.

### Algoritmus

Algoritmus tedy vypadá následovně:

**Inicializace**
- Nastavíme vhodné počáteční hyperparametry apriorního normálního rozdělení $\widehat\beta_{t-1}, \Sigma_{t-1}$.

**Online modelování**
1. Získáme nová měření $y_t$ a regresor $x_t$.
2. Napočítáme MAP odhad $\widehat{\beta}_t$.
3. Napočítáme aproximativní kovarianci odhadu $\Sigma_t$.

### Predikce

Bayesovská predikce běží přes prediktivní distribuci, která ovšem rovněž není analyticky dostupná. Místo toho můžeme využít laplaceovskou aproximaci

$$
\begin{aligned}
f(y'|x', x_{0:t}, y_{0:t}) 
&= \int f(y'|x', x_{0:t}, y_{0:t}, \beta) \pi(\beta|x_{0:t}, y_{0:t}) d\beta  \\
&\approx
(2\pi)^{\frac{n}{2}} \left[\det l''(\widehat{\beta}_t)\right]^{-\frac{1}{2}} f(y'|x', \beta)\ \pi(\beta|x_{0:t},y_{0:t}),
\end{aligned}
$$

kde dosadíme $\beta = \widehat{\beta}_t$.